In [1]:
import numpy as np
import pandas as pd

data = pd.read_csv("DN4_1_podatki.csv")
X = data.drop("Q", axis=1)
y = data[["Q"]]

In [2]:
data

,Q,Tw,Ta,theta,eta
0,1.711929,53.005339,10.788147,0.321921,0.349083
1,0.067135,20.761340,12.225502,2.705824,0.821069
2,0.034759,39.546739,35.892183,1.873776,0.756082
3,0.417783,33.972185,21.385801,2.089589,0.516134
4,1.667614,50.092173,15.254621,0.328350,0.083327
...,...,...,...,...,...
495,1.327393,54.081114,30.039247,1.153132,0.708757
496,0.053967,42.933431,39.799381,1.572222,0.022975
497,0.707523,47.145488,30.707064,2.093919,0.684035
498,0.062350,39.405105,32.647363,0.311168,0.053085


In [3]:
from pysr import PySRRegressor



In [4]:
model = PySRRegressor(
    niterations=50,  # < Increase me for better results
    #populations=30,
    #population_size=50,
    # ^ Slightly larger populations, for greater diversity.
    binary_operators=["+", "-", "*", "/", "^"],
    unary_operators=[
        "cos",
        "sin",
        "square",
        "cube",
        "neg",
        "inv(x)=1/x"
    ],
    extra_sympy_mappings={"inv": lambda x: 1 / x},
    hof_migration=False,
    temp_equation_file="halls",
    tempdir="halls",
    nested_constraints={
        "cos" : {"cos" : 0, "sin" : 0},
        "sin" : {"cos" : 0, "sin" : 1}
    },
    #timeout_in_seconds=60,
    #complexity_of_constants=0.2,
    #weight_randomize=0.1,
    #maxsize=30,
    maxdepth=9,
    procs=0,
    multithreading=False,
    warm_start=False,
    random_state=0,
    deterministic=True
)
#model.reset()

model.fit(X,y)

/home/tadej/programming/nsu/dn4/.venv/lib/python3.11/site-packages/pysr/sr.py:1296: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
/home/tadej/programming/nsu/dn4/.venv/lib/python3.11/site-packages/pysr/sr.py:1364: FutureWarning: Index.is_object is deprecated.Use pandas.api.types.is_object_dtype instead
  if X.columns.is_object() and X.columns.str.contains(" ").any():


Started!

Expressions evaluated per second: 4.910e+04
Progress: 122 / 750 total iterations (16.267%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
1           1.150e+00  1.348e-06  0.6827156
2           1.026e+00  1.144e-01  sin(theta)
4           1.022e+00  2.010e-03  (sin(theta) / 0.9168521)
6           6.392e-01  2.344e-01  square((-0.42600712 / Ta) * Tw)
8           6.391e-01  5.848e-05  square(sin(neg(-0.42600712) / Ta) * Tw)
---------------------------------------------------------------------------------------------------
Press 'q' and then <enter> to stop execution early.

Expressions evaluated per second: 4.120e+04
Progress: 188 / 750 total iterations (25.067%)
Hall of Fame:
---------------------------------------------------------------------------------------------------
Complexity  Loss       Score     Equation
1           1.150e+00  1.348e-06  0.6827156
2          

PySRRegressor.equations_ = [
	    pick     score                                           equation   
	0         0.000000                                          0.6827156  \
	1         0.114404                                         sin(theta)   
	2         0.015706                                  (0.01904036 * Tw)   
	3         0.040474                                 cube(cos(Ta / Tw))   
	4         0.376170                    square((-0.42600712 / Ta) * Tw)   
	5         0.012367                   (sin(sin(theta)) / inv(Tw / Ta))   
	6         0.424400                (sin(theta) / inv((Tw / Ta) - eta))   
	7         0.035121  ((sin(theta) / inv((Tw / Ta) - eta)) ^ 1.0788765)   
	8         0.011761  (sin(sin(theta)) / (inv((Tw / Ta) - 0.64234614...   
	9         0.082738  inv(cube(Ta ^ inv((Tw - Ta) - -0.72189254)) - ...   
	10        0.205963  cube(cube(cos(Ta / (Tw + -12.117218))) - (squa...   
	11        0.125090  inv(square(cube(Tw ^ inv((Tw - Ta) - -0.344796...   
	12        0.246734  inv(square(cube((Tw + Tw) ^ inv((Tw - Ta) - -0...   
	13  >>>>  0.207012  inv(square(cube((Tw + (Tw * eta)) ^ inv((Tw - ...   
	14        0.001660  inv(square(cube((Tw + ((Tw + 0.36919403) * eta...   
	
	        loss  complexity  
	0   1.149929           1  
	1   1.025619           2  
	2   1.009637           3  
	3   0.931129           5  
	4   0.639207           6  
	5   0.623590           8  
	6   0.407929           9  
	7   0.380258          11  
	8   0.375812          12  
	9   0.345969          13  
	10  0.281572          14  
	11  0.248464          15  
	12  0.194137          16  
	13  0.128322          18  
	14  0.127896          20  
]

In [5]:
print(model.score(X,y))
model.sympy()

0.8884092886152661


((Tw*eta + Tw)**(3/(-Ta + Tw + 0.34479672)) - sin(theta))**(-2)

In [6]:
model.score(X,y)

0.8884092886152661

In [7]:
model.predict(X)

array([9.33933624e-01, 1.11910843e-01, 1.87160737e-03, 3.77757442e-01,
       8.52197713e-01, 2.60810827e+00, 1.11661141e+00, 4.64737039e-01,
       7.69961776e-01, 1.52997486e+00, 7.22503854e-01, 1.50757857e-01,
       3.24992172e-01, 4.15736727e-01, 1.91213748e-01, 6.45241498e-01,
       5.80914605e-01, 6.46878024e-01, 1.14992115e-03, 1.82248005e-01,
       2.53496523e-10, 2.76531547e+00, 8.34383122e-01, 4.24157751e-03,
       1.08033427e+00, 1.24019803e-06, 1.01153042e-02, 6.01288933e-01,
       1.50095287e-01, 1.30356758e+00, 7.05978233e-01, 1.92045911e-07,
       5.35596657e-01, 1.56025574e-04, 1.04765276e-01, 2.48837900e-05,
       3.42917134e-01, 5.77649138e-01, 1.07762907e-01, 1.07316131e+00,
       2.08712602e-02, 1.57058493e-16, 1.18536667e+00, 7.58695806e-01,
       2.56054378e-01, 1.48731591e+00, 1.21099549e-01, 4.43997948e-05,
       5.28376743e-01, 8.55691179e-01, 1.18479256e-01, 1.29532997e+00,
       1.35609960e+00, 1.26347264e-02, 5.59783601e-01, 9.77502367e-02,
      